Intro, what is GRAF, why NASLib wrapper, main features

In [ ]:
"""
# clone a NASLib fork that supports python 3.11
!git clone -b automl-conf-24 --single-branch git@github.com:gabikadlecova/NASLib.git

# install NASLib
%cd NASLib
%pip install -e .
# %pip install ConfigSpace==0.6 --quiet
%cd ..
"""

# download dataset apis and NAS-Bench-Suite Zero data
"""
%cd /content/NASLib
!source scripts/bash_scripts/download_benchmarks.sh nb201 cifar10
# !source scripts/bash_scripts/download_benchmarks.sh nb201 cifar100
# !source scripts/bash_scripts/download_benchmarks.sh nb201 ImageNet16-120
# !source scripts/bash_scripts/download_benchmarks.sh nb301 cifar10
# !source scripts/zc/bash_scripts/download_nbs_zero.sh nb101
!source scripts/zc/bash_scripts/download_nbs_zero.sh nb201
# !source scripts/zc/bash_scripts/download_nbs_zero.sh nb301
"""

In [ ]:
"""
!git clone git@github.com:gabikadlecova/GRAF.git
%cd GRAF
%pip install -e .
%cd ..
"""

## Search spaces

In [1]:
# ------ Setup ------ #
benchmark = 'nb201'
dataset = 'cifar10'

feature_cfg = '../graf_nas/configs/nb201.json'

naslib_path = '../../zero_cost/NASLib'

In [2]:
from graf_nas.search_space import NB201

net = '(2, 0, 0, 2, 0, 2)'  # a network with 3 sequential 3x3 conv-bn-relu, and skip-connections between them

# unified representation of architectures from a search space
network = NB201(net, cache_model=True)
print("Network: ", network.net)

# the network can be converted to a graph
graph = network.to_graph()
print("Edges: ", graph.edges)
print("Ops: ", graph.ops)
print("Networkx graph: ", graph.to_graph())


Network:  (2, 0, 0, 2, 0, 2)
Edges:  {(1, 2): 2, (1, 3): 0, (1, 4): 0, (2, 3): 2, (2, 4): 0, (3, 4): 2}
Ops:  {0: 'skip_connect', 1: 'none', 2: 'nor_conv_3x3', 3: 'nor_conv_1x1', 4: 'avg_pool_3x3'}
Networkx graph:  DiGraph with 4 nodes and 6 edges


In [3]:
import torch

# you can convert the network to a parsed NASLib object
model = network.get_model()
print(model)

# it is a PyTorch model
print("Is it torch.nn.Module: ", isinstance(model, torch.nn.Module))

model(torch.randn(1, 3, 32, 32))  # you can directly pass data to the model

NasBench201SearchSpace named 'makrograph' with 20 nodes and 19 edges
Is it torch.nn.Module:  True


tensor([[ 0.0585,  0.3060,  0.2005, -0.1932,  0.2344,  0.6175, -0.4376, -0.1304,
          0.1565,  0.0720]], grad_fn=<AddmmBackward0>)

In [4]:
from graf_nas.search_space import searchspace_classes, dataset_api_maps
from naslib.utils import get_dataset_api

# you can also get the NB201 class by indexing the searchspace_classes dictionary
net_cls = searchspace_classes[benchmark]

# the class itself supports iterating over all architectures in the search space (you need to download NASLib dataset api - done at the start of the doc)
dataset_api = get_dataset_api(search_space=dataset_api_maps[benchmark], dataset=dataset)
net_iterator = net_cls.get_arch_iterator(dataset_api)

next(iter(net_iterator)).net

'(0, 0, 0, 0, 0, 0)'

## zero-cost proxies

In [5]:
# GRAF support easy computation of all zero-cost proxies in NAS-Bench-Suite Zero (and also extension with new ones)
import os
from graf_nas.features.zero_cost import get_zcp_predictor, get_zcp_dataloader, available_measures

print('Available NASLib zero-cost proxies: ', available_measures)

# You need to download the data loader for ZCP computation
dataloader = get_zcp_dataloader(dataset=dataset, zc_cfg=os.path.join(naslib_path, 'naslib/runners/zc/zc_config.yaml'), data=naslib_path)

# You can get a predictor for a specific zero-cost proxy
predictor = get_zcp_predictor('synflow', dataloader=dataloader)
print(predictor)

# using the PyTorch model from previous cells, compute the synflow score! (you might want to change to the GPU runtime)
print(predictor(model))

Available NASLib zero-cost proxies:  ['grad_norm', 'snip', 'grasp', 'fisher', 'jacov', 'plain', 'synflow_bn', 'synflow', 'epe_nas', 'zen', 'l2_norm', 'nwot']
Files already downloaded and verified
Files already downloaded and verified
synflow
111.83579934901714


## Neural Graph features

In [6]:
import json
import pprint

# represented by a config
with open(feature_cfg, 'r') as f:
    cfg = json.load(f)

# This config contains 4 features - operation counts, min path length, max path length, and node degrees
pprint.pprint(cfg, sort_dicts=False)

[{'name': 'op_count'},
 {'name': 'min_path_len',
  'allowed': ['skip_connect',
              'none',
              'nor_conv_3x3',
              'nor_conv_1x1',
              'avg_pool_3x3']},
 {'name': 'max_op_on_path',
  'allowed': ['skip_connect',
              'none',
              'nor_conv_3x3',
              'nor_conv_1x1',
              'avg_pool_3x3']},
 {'name': 'node_degree',
  'allowed': ['skip_connect',
              'none',
              'nor_conv_3x3',
              'nor_conv_1x1',
              'avg_pool_3x3']}]


In [20]:
from graf_nas.features.config import load_from_config

# the features from the config are loaded as a list of objects
features = load_from_config(cfg, benchmark)
print(features[:2])
print()

# these represent graf features limited to a subset of the operation set
print([str(f) for f in features[:10:2]])

[<graf_nas.features.base.Feature object at 0x7facb9de1990>, <graf_nas.features.base.ConstrainedFeature object at 0x7faca756d210>]

['op_count', 'min_path_len_(none)', 'min_path_len_(nor_conv_1x1)', 'min_path_len_(skip_connect,none)', 'min_path_len_(skip_connect,nor_conv_1x1)']


In [30]:
some_feat = features[1]

print(f"Computing {some_feat} for the following network:")
print({k: graph.ops[v] for k, v in graph.edges.items()})
print()

print("Result: ", some_feat(graph))  # you can compute the features for a network graph

Computing min_path_len_(skip_connect) for the following network:
{(1, 2): 'nor_conv_3x3', (1, 3): 'skip_connect', (1, 4): 'skip_connect', (2, 3): 'nor_conv_3x3', (2, 4): 'skip_connect', (3, 4): 'nor_conv_3x3'}

Result:  1


## GRAF - easy computation of ZCP and features

In [ ]:
# TODO load ZCPs in DataFrame / via func in NB Suite Zero